# Trying to get the completness of the Web Scarper

In [47]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()


#AzureOpenAI setup
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    # api_type='azure',
    api_version='2023-05-15',  # this may change in the future
    timeout=20*60,  # 20 minutes
)

#To generate summaries from 1000's of comments
def generate_summary(prompt):
    response = client.chat.completions.create(
        temperature=0.0,
        model="gpt-4-1106-preview",
        messages=[
                {"role": "system", "content":"""You are a validation agent.
                """},
                {"role": "user", "content": prompt },
            ]
    )

    return response.choices[0].message.content

## The three cells below load the text files. 
1. Trafilatura scraped text
2. BeautifulSoup Scraped text
3. A manual scraped text file, where I manually go into the website and copy and paste content

In [1]:
#opening the manually created text file

txt_file_path = 'scraped_data/TenetHealth_trafilatura.txt'

with open(txt_file_path, 'r') as file:
    text_trafilatura = file.read()


In [2]:
txt_file_path = 'scraped_data/TenetHealth_BeautifulSoup.txt'

with open(txt_file_path, 'r') as file:
    text_BSoup = file.read()


In [4]:
txt_file_path = 'manual_scraping/TenetHealth.txt'

with open(txt_file_path, 'r') as file:
    text_manual = file.read()


## Defining the Prompt

#### Comparing manual text and BeautifulSoup

In [39]:
prompt = """You are validation agent helping to validate if the given text and the generated text have the same content. 
            Return the a score ranging from 1 to 100 of how accurate the given text is to the generated text, make sure the content is the same.
            Also mention the difference between the texts in less than 200 words, if they exist""" + f'"""Given Text: {text_manual}"""' + f'"""Generated Text: {text_BSoup}"""'

In [40]:
summary = generate_summary(prompt)
print(summary)

#### Comparing manual text and Trafilatura

In [45]:
prompt2 = """You are validation agent helping to validate if the given text and the generated text have the same content. 
            Return the a score ranging from 1 to 100 of how accurate the given text is to the generated text, make sure the content is the same.
            Also mention the difference between the texts in less than 200 words, if they exist""" + f'"""Given Text: {text_manual}"""' + f'"""Generated Text: {text_trafilatura}"""'

In [46]:
summary2 = generate_summary(prompt2)
print(summary2)

Accuracy Score: 100

The given text and the generated text are identical, with no differences between them. The generated text is a direct copy of the source material provided from the Tenet Healthcare website, including various sections such as responding to COVID-19, terms and conditions of use, our stories, who we are, news, diversity & inclusion, locations, careers, Tenet through the years, improving our impact on the environment, contact us, giving back, what we do, ethics & compliance, sitemap, our leadership, notice of privacy practices, our locations, privacy policy, accessibility statement, programs & benefits, join our team, community outreach, search our careers, and physician careers.


# Notes:
1. While the completness accuracy is 100 for both, the formatting style is a bit different.
2. Due to difference in formating style the token usage is different (BSoup > Trafilatura).
3. For better results, I have converted these text files into vector indices and ran RAG evaluation. (Found in RAG_Eval notebook) 